In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import dagshub
import pickle

dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [2]:

# Step 1: Clean and isolate features and target
df = df.dropna(subset=['category'])
X = df['clean_comment']
y = df['category']

# Step 2: Perform the train-test split FIRST to prevent data leakage 💧
# 'stratify=y' ensures the original class proportions are maintained in both sets.
X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Fit the TF-IDF vectorizer ONLY on the training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train_text)

# Use the fitted vectorizer to transform the test data
X_test_vec = vectorizer.transform(X_test_text)

# Step 4: Apply SMOTE to the TRAINING data ONLY to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# The test data (X_test_vec, y_test) remains untouched for a realistic evaluation.

# --- MLflow and Optuna Functions (no changes needed) ---

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams_Corrected")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        model_filename = f"{model_name}_model.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_filename, "model")

# Step 5: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    # Train on the resampled training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Evaluate on the original, untouched test data
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 6: Run Optuna for SVM
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    best_params = study.best_params
    best_model = SVC(**best_params, random_state=42)
    
    # Log the best model using the correctly processed datasets
    log_mlflow("SVM", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment
run_optuna_experiment()

[I 2025-08-27 20:25:44,750] A new study created in memory with name: no-name-c63f697d-bd65-4df4-b777-0ddbf6c1a3f3
[I 2025-08-27 20:44:18,554] Trial 0 finished with value: 0.49703407366533314 and parameters: {'C': 0.0004423440906911965, 'kernel': 'linear'}. Best is trial 0 with value: 0.49703407366533314.
[I 2025-08-27 21:14:12,122] Trial 1 finished with value: 0.7683818457718306 and parameters: {'C': 4.9131214507011105, 'kernel': 'rbf'}. Best is trial 1 with value: 0.7683818457718306.
[I 2025-08-27 21:23:46,866] Trial 2 finished with value: 0.45095875293143883 and parameters: {'C': 0.0010483332479892429, 'kernel': 'poly'}. Best is trial 1 with value: 0.7683818457718306.
[I 2025-08-27 21:30:09,557] Trial 3 finished with value: 0.772658297696234 and parameters: {'C': 0.27322010220616466, 'kernel': 'linear'}. Best is trial 3 with value: 0.772658297696234.
[I 2025-08-27 21:39:16,566] Trial 4 finished with value: 0.4909642709339219 and parameters: {'C': 0.029443498149364988, 'kernel': 'rbf'

🏃 View run SVM_SMOTE_TFIDF_Trigrams_Corrected at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/d85512cf83354e58bed8f3253f5aab78
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
